In [18]:
# clean_strclean_stringgput_latex = "For all $x$ in $$(3,5)$$\n there is an $n$ in $$(4,\\infty)$$\n\\textbf{Note:} \\$ should display a dollar sign.  \\emph{blabla} should be emphasized.  \\href{google.com}{A website}"
# input_latex = "For all $x$ in $$(3,5)$$ with $x<4.25$, there is an $n$>3 and <8 in $$(4,\\infty)$$.  But \\$ is a dollar sign."
# input_latex = "\\section{New section!}\\textbf{Note:}\\emph{blabla}\\href{google.com}{A website}"
# input_latex = "For all $x$ in $$(3,5)$$\n there is an $n$ in $$(4,\\infty)$$\n\\section{New section!}\n\\textbf{Note:} \\$ should display a dollar sign.  \\emph{blabla} should be emphasized.  \\href{google.com}{A website}"
# input_latex = "\\section{New $\\epsilon$ section!}\\textbf{Note:}\\emph{blabla $f(x) = 5x+3$}\\href{google.com}{A website}"
# input_latex = "\\section{This is some text} Here is body text \\subsection{subdivision} more text \\section{New section} with text"
# input_latex = "\\section{This is some text} Here is body text \\begin{enumerate} \\item Hello \\item How \\item Are \\item You? \\end{enumerate}\\subsection{subdivision} more text \\section{New section} with text"
# input_latex = "\\begin{enumerate}\\item Level one \\begin{itemize}\\item have cake \\item eat it too \\end{itemize} \\item Lorem Ipsem \\end{enumerate}"
# input_latex = "lala\\textbf{\\begin{enumerate} \\item $5x-1$ \\item raca\\end{enumerate} }lala"
input_latex = "\\textbf{\\begin{enumerate} \\item $5x-1$ \\item raca\\end{enumerate}}"

# input_latex = input("Please type some LaTeX here")

# print("Input LaTeX:", input_latex)
# print("==============")

def split_arguments(argument_string):
    output = []
    current_string = ""
    for char in argument_string:
        if char == "{" or char == "[":
            pass
        elif char == "}" or char == "]":
            output += [current_string]
            current_string = ""
        else:
            current_string += char
    return output

def clean_string(str):
    output = ""
    for char in str: 
        if char == "<":
            output  += "&lt;"
        elif char == ">":
            output  += "&gt;"
        elif char == "&":
            output += "&amp;"
        else:
            output  += char
    return output

# Latex macros of ONE argument that convert to <env>input</env>
text_formatting = {
        "textbf" :  "alert", 
        "textit" :  "alert", 
        "emph"   :  "alert",
    } 
# Latex macros with at least one argument that becomes an attribute.  
# the second argument is used to fill in the contents of the environment.
environment_with_attribute = {
        "href" : {"environment":"url", "attribute":"href", "content_allowed" : True },
        "cite" : {"environment":"xref", "attribute":"ref", "content_allowed" : False},
        "ref"  : {"environment":"xref", "attribute":"ref", "content_allowed" : False},
    }   
list_environments = { "enumerate":"ol", "itemize":"ul", "description":"dl"}
ignore = ["vspace","hspace","bigskip","smallskip","medskip","bigspace","newpage"]

# Known divisions and whether they are currently in use
latex_divisions = {"chapter":False,"section":False,"subsection":False,"subsubsection":False}
division_priority = ["subsubsection","subsection","section","chapter"]

known_escaped_latex = {"%":"%","$":"$","{":"{","}":"}","_":"_","#":"#","&":"&amp;","-":""}

previous_backslash = False
reading_argument = False
in_argument = False
macro_name = ""
macro_argument_string = ""

macro_stack = []

math_mode = False
display_math = False
previous_dollar = False
exiting_display = False
active_double_dollar = False

# TODO: implement lists, figures, tables, etc
# Ideally, do this using a dictionary 

debugging=False

# Add a single character at the end to force the loop to be run one last time after the last character is read
input_latex = input_latex + " "
output_plaintext = ""


def process_text(input_latex):

    input_latex_array = list(input_latex)
    # Execution of some tasks (math mode, macros) takes place 
    # when the NEXT character is read.  This causes bugs when 
    # the last character is important (e.g. a closing })
    # Fix this by adding a single empty character to the end 
    # of the array of characters we are processing.
    input_latex_array += [""]

    # Global Dictionaries for substitution
    global text_formatting
    global environment_with_attribute
    global list_environments
    global ignore
    global latex_divisions
    global division_priority
    global known_escaped_latex
    # Global stack of macros
    global macro_stack
    #
    global math_mode
    global display_math
    global previous_dollar
    global exiting_display
    global active_double_dollar
    # 
    # global previous_backslash
    # global reading_argument
    # global in_argument
    # global macro_name
    # global macro_argument_string
    previous_backslash = False
    reading_argument = False
    in_argument = False
    macro_name = ""
    macro_argument_string = ""
    delimiter_stack = []
    split_argument_array = []
    # global output_plaintext
    output_plaintext=""
    

    
    while not(len(input_latex_array)==0):
        char = input_latex_array.pop(0)

        if debugging: print(macro_stack) 

        if char == "%" : 
            print("WARNING: Comments not implemented yet. Copy/pasting not obviously inserting newline symbols.")

        if not(reading_argument) and  (math_mode or display_math):
            if char == "$":
                if previous_dollar:# entering display math
                    output_plaintext += "<md>"
                    math_mode = False
                    display_math = True
                    previous_dollar = False
                elif math_mode:# exiting regular math
                    output_plaintext += "</m>"
                    math_mode = False
                elif display_math and not(exiting_display):
                    exiting_display = True
                elif display_math and exiting_display:
                    output_plaintext += "</md>"
                    display_math = False
                    exiting_display = False

            elif previous_dollar: # Just entered regular math mode
                previous_dollar = False
                output_plaintext += "<m>"
                # output_plaintext += clean_string(char)
                output_plaintext +=  char

            else:
                # output_plaintext += clean_string(char)
                output_plaintext +=  char

        elif previous_backslash:

            if macro_name=="" and char == '\\' :
                if len( macro_stack)>0 and macro_stack[-1]=="p":
                    output_plaintext +=  "</p>\n<p>"
                else:
                    output_plaintext += "<p>"
                #TODO: Implement forced newline properly with <p>
                previous_backslash = False
        
            elif macro_name=="" and char in known_escaped_latex :

                # TODO: Open paragraph if needed

                output_plaintext += known_escaped_latex[char]
                previous_backslash = False

            elif char.isalpha():

                # TODO: Open paragraph if needed

                # macro_name += clean_string(char)
                macro_name += char
                # if debugging: print(macro_name)

            elif not(macro_name=="") and (char == "{" or char =="[" ):
                delimiter_stack += [char]
                previous_backslash = False 
                reading_argument = True
                in_argument = True
                # macro_argument_string += clean_string(char)
                # macro_argument_string += char

            else: ## A latex macro has ended without any arguments.

                # TODO: Open paragraph if needed

                previous_backslash = False
                
                if macro_name in ignore:
                    pass

                elif macro_name == "item":
                    ## TODO: check macro_stack length before examining last element here and elsewhere
                    # if len(macro_stack)>0:

                    if macro_stack[-1] == "p":
                        output_plaintext += "</p>\n"
                        macro_stack.pop()

                    if macro_stack[-1] == "li":
                        output_plaintext += "</li>\n<li>\n<p>"
                        macro_stack += ["p"]
                    else:
                        output_plaintext += "<li>\n<p>"
                        macro_stack += ["li"]
                        macro_stack += ["p"]
                elif macro_name == "par":
                    if len(macro_stack)>0 and macro_stack[-1]=="p":
                        output_plaintext += "</p>\n<p>"
                    else:
                        output_plaintext += "<p>"
                        macro_stack += ["p"]
                else:
                ## Take action 
                    output_plaintext = output_plaintext + "**" + macro_name + "**"

                macro_name = ""

                ## TODO: Deal with the new character (in case mathmode, another macro, etc start immediately)
                if char == "\\":
                    previous_backslash=True
                elif char == "$":

                    # TODO: Open paragraph if needed

                    previous_dollar = True
                    math_mode = True
                else:
        
                    # TODO: Open paragraph if needed

                    # output_plaintext += clean_string(char)
                    output_plaintext += char

        elif reading_argument:

            # print(char)
            # print(macro_argument_string)
            # print(delimiter_stack)

            # TODO: error checking  
            
            ## handle mathmode inside arguments 
            ## using code for math environment copied from non-argument mode
            if in_argument and not(char == "}" or char == "]" ):
                if math_mode or display_math:
                    if char == "$":
                        if previous_dollar:# entering display math
                            macro_argument_string += "<md>"
                            math_mode = False
                            display_math = True
                            previous_dollar = False
                        elif math_mode:# exiting regular math
                            macro_argument_string += "</m>"
                            math_mode = False
                        elif display_math and not(exiting_display):
                            exiting_display = True
                        elif display_math and exiting_display:
                            macro_argument_string += "</md>"
                            display_math = False
                            exiting_display = False
                    elif previous_dollar: # Just entered regular math mode
                        previous_dollar = False
                        macro_argument_string += "<m>"
                        # macro_argument_string += clean_string(char)
                        macro_argument_string += char
                    else:
                        # macro_argument_string += clean_string(char)
                        macro_argument_string += char

                elif char == "$":
                    previous_dollar = True
                    math_mode = True
                elif char == "{" or char == "[":
                    if len(delimiter_stack)>0 : 
                        macro_argument_string += char
                    delimiter_stack += [char]
                    # macro_argument_string += clean_string(char)
                # elif char =="/":
                #     TODO: support (reasonable) macros in arguments
                #     PROBLEM: nested arguments will require separate 'in_argument' flags
                else:
                    macro_argument_string += char
                    # macro_argument_string += clean_string(char)
                    # if debugging: print(macro_argument_string)

            elif in_argument and ( char == "}" or char == "]" ):

                if len(delimiter_stack)>0:
                    delimiter_stack.pop()
                else: print("Error: mismatch in delimiter stack")

                # macro_argument_string += clean_string(char)
                
                if len(delimiter_stack) == 0:
                    in_argument=False
                    # macro_argument_string += " "
                    split_argument_array += [macro_argument_string]
                    macro_argument_string = ""
                else:
                    macro_argument_string += char

                # if debugging: print(macro_argument_string)

            elif char == "{" or char == "[":
                if len(delimiter_stack)>0:
                    macro_argument_string += char

                delimiter_stack += [char]
                in_argument=True
                # macro_argument_string += clean_string(char)
            
            else: # Done reading macro and argument
                
                in_argument = False
                reading_argument = False
                
                # TODO: Open paragraph if needed

                if debugging: print("parsed macro:",macro_name,split_argument_array)

                ## TODO: call process_text to interpret the argument array
                # processed_argument_string = process_text(macro_argument_string)
                # print(processed_argument_string)
                # split_argument_array = split_arguments(processed_argument_string) 
                processed_arg_array = [process_text(argument) for argument in split_argument_array]
                split_argument_array = []


                # split_argument_array = split_arguments(macro_argument_string) 


                if macro_name in ignore:
                    pass

                elif macro_name in text_formatting:
                    env = text_formatting[macro_name]
                    output_plaintext += "<"+env+">"+processed_arg_array[0]+"</"+env+">"

                elif macro_name in environment_with_attribute:
                    if environment_with_attribute[macro_name]["content_allowed"]:
                        ptx_environment = environment_with_attribute[macro_name]["environment"]
                        ptx_attribute = environment_with_attribute[macro_name]["attribute"]
                        if len(processed_arg_array)>1:
                                output_plaintext += "\n<"+ptx_environment+" "+ptx_attribute+'="'+split_argument_array[0]+'">' 
                                output_plaintext += processed_arg_array[1]
                                output_plaintext += "\n</" + ptx_environment + ">"
                        else:
                            output_plaintext += "\n<"+ptx_environment+" "+ptx_attribute+'="'+split_argument_array[0]+'" />'

                    else:
                        output_plaintext += "\n<"+ptx_environment+" "+ptx_attribute+'="'+split_argument_array[0]+'" />'

                elif macro_name == "begin":
                    env = processed_arg_array[0]
                    if len(processed_arg_array)==0:
                        print("Error: begin has no argument")

                    if env in list_environments:
                        env = list_environments[env] 

                    macro_stack += [env]
                    output_plaintext += "\n<" + env + ">\n"

                    if debugging: print(macro_stack)

                    #TODO: implement regular environments
                    #TODO: error checking using stack of environments
                    #TODO: For enumerate, itemize, description, closing tags should also include </li>
            
                elif macro_name == "end":
                    if debugging: print(macro_stack)
                    env = processed_arg_array[0]
                    if len(processed_arg_array)==0:
                        print("Error: end has no argument")

                    if macro_stack[-1] == "p":
                        if debugging: print("popping par")
                        output_plaintext += "</p>\n"
                        popped_macro = macro_stack.pop()
                        if not(popped_macro == "p"): print("Error: mismatched popped macro",popped_macro)

                    if env in list_environments:

                        if macro_stack[-1] == "p":
                            output_plaintext += "</p>\n"
                            macro_stack.pop()

                        env = list_environments[env]
                        output_plaintext += "</li>"
                        if debugging: print("popping li")
                        popped_macro = macro_stack.pop()
                        if not(popped_macro == "li"): print("Error:: mismatched popped macro",popped_macro)

                    popped_macro = macro_stack.pop()
                    if not(popped_macro == env): print("Error::: mismatched popped environment",popped_macro,env)
                    if debugging: print(macro_stack)
                    output_plaintext += "\n</" + env + ">\n"

                elif macro_name == "item":
                    argument = processed_arg_array[0]
                    # if len(macro_stack)>0:

                    if macro_stack[-1] == "p":
                        output_plaintext += "</p>\n"
                        macro_stack.pop()

                    if macro_stack[-1] == "li":
                        output_plaintext += "</li>\n<li>\n<title>"+argument+"</title>\n<p>"
                        macro_stack.append("p")
                    else:
                        output_plaintext += "<li>\n<title>"+argument+"</title>\n<p>"
                        macro_stack += ["li"]
                        macro_stack += ["p"]

                elif macro_name in latex_divisions:
                    
                    for div in division_priority:
                        if debugging: print("testing",div,latex_divisions[div])

                        if len(macro_stack)>0 and macro_stack[-1] == "p":
                            output_plaintext += "</p>\n"
                            popped_macro = macro_stack.pop()
                            if not(popped_macro == "p"): print("Error: mismatched popped macro",popped_macro)

                        if latex_divisions[div]:
                            output_plaintext += "</" + div + ">" 
                            popped_macro = macro_stack.pop()
                            if not(div==popped_macro ): print("Warning:",div,popped_macro)
                            latex_divisions[div]=False

                        if div == macro_name:
                            latex_divisions[macro_name]=True
                            macro_stack.append(macro_name)
                            if debugging: print(macro_stack)
                            output_plaintext += "<" + macro_name + ">\n<title>" + split_argument_array[0] + "</title>\n"
                            break

                    # if latex_divisions[macro_name]:
                    #     output_plaintext += "</" + macro_name + ">" 
                    #     output_plaintext += "<" + macro_name + ">\n<title>" + split_argument_array[0] + "</title>\n"
                    # else:
                    #     latex_divisions[macro_name] = True
                    #     macro_stack.append(macro_name)
                    #     output_plaintext += "<" + macro_name + ">\n<title>" + split_argument_array[0] + "</title>\n"

                    macro_name = ""
                    macro_argument_string = ""

                else:
                    for argument in processed_arg_array:
                        output_plaintext += macro_name + macro_argument_string
            
                # Deal with whatever character we actually just read
                if char =="\\":
                    previous_backslash = True
                elif char =="$":
                    math_mode = True
                    previous_dollar = True
                else:
                    # output_plaintext += clean_string(char)
                    output_plaintext += char
                macro_name = ""
                macro_argument_string = ""

        elif char == "\\" and not(previous_backslash):
            previous_backslash = True

        elif char == "$": 
            math_mode = True
            previous_dollar = True

        else: 
            # if char == "<"  :
            #     output_plaintext += "&lt;"
            # elif char == ">":
            #     output_plaintext += "&gt;"
            # elif char == "&":
            #     output_plaintext += "&amp;"
            # else:
            #     # output_plaintext += clean_string(char)
            output_plaintext += char
    
    return output_plaintext


## cleanup any forgotten open environments.
if math_mode:
    output_plaintext += "</m>"
elif display_math:
    output_plaintext += "</md>"

## Automatically close open divisions
if latex_divisions["subsubsection"]: 
    output_plaintext += "</subsubsection>"
    current_division = macro_stack.pop()
    if not(current_division == "subsubsection"): 
        print("Macro stack error")
    active_subsection = False
elif latex_divisions["subsection"]: 
    output_plaintext += "</subsection>"
    current_division = macro_stack.pop()
    if not(current_division == "subsection"): 
        print("Macro stack error")
    active_subsection = False
elif latex_divisions["section"]: 
    output_plaintext += "</section>"
    current_division = macro_stack.pop()
    if not(current_division == "section"): 
        print("Macro stack error")
    active_section = False
elif latex_divisions["chapter"]: 
    output_plaintext += "</chapter>"
    current_division = macro_stack.pop()
    if not(current_division == "chapter"): 
        print("Macro stack error")
    active_section = False


## Sanitize ONCE at the beginnnig
## Do NOT sanitize in recursive function (must preserve xml tags)
input_latex = clean_string(input_latex)
output_plaintext=process_text(input_latex)

print( output_plaintext)



<alert>
<ol>
 <li>
<p> <m>5x-1</m> </p>
</li>
<li>
<p> raca</p>
</li>
</ol>

</alert> 
